# GRU Models for Sarcasm Detection

## Project Overview
This notebook implements GRU (Gated Recurrent Unit) models for sarcasm detection using both PyTorch and TensorFlow frameworks. The task involves:

1. **Binary Classification**: Detecting sarcasm in news headlines
2. **Deep Learning Models**: GRU-based architectures in PyTorch and TensorFlow
3. **Evaluation Metrics**: Accuracy, Precision, Recall, F1-Score, AUC, ROC
4. **Hyperparameter Tuning**: Using Keras Tuner for TensorFlow model optimization
5. **Target Performance**: Minimum 80% accuracy on both training and testing sets

## Dataset
- **Source**: DeteksiSarkasme.json
- **Features**: News headlines
- **Target**: Binary classification (0: Not Sarcastic, 1: Sarcastic)

In [ ]:
# Import Required Libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Text Processing
import re
import string
from collections import Counter
from wordcloud import WordCloud

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Keras Tuner for hyperparameter tuning
try:
    import keras_tuner as kt
    KERAS_TUNER_AVAILABLE = True
except ImportError:
    print("Keras Tuner not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'keras-tuner'])
    import keras_tuner as kt
    KERAS_TUNER_AVAILABLE = True

# Scikit-learn for metrics and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.preprocessing import LabelEncoder

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
tf.random.set_seed(42)

# Check for GPU availability
print("GPU Available (PyTorch):", torch.cuda.is_available())
print("GPU Available (TensorFlow):", tf.config.list_physical_devices('GPU'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
data_path = '../Dataset/DeteksiSarkasme.json'

# Read JSON data line by line
data = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

# Convert to DataFrame
df = pd.DataFrame(data)

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

print("\nColumn names:")
print(df.columns.tolist())

print("\nTarget distribution:")
print(df['is_sarcastic'].value_counts())
print("\nTarget distribution (percentage):")
print(df['is_sarcastic'].value_counts(normalize=True) * 100)

In [ ]:
# Exploratory Data Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Target distribution
df['is_sarcastic'].value_counts().plot(kind='bar', ax=axes[0,0], color=['skyblue', 'orange'])
axes[0,0].set_title('Sarcasm Distribution')
axes[0,0].set_xlabel('Is Sarcastic')
axes[0,0].set_ylabel('Count')
axes[0,0].set_xticklabels(['Not Sarcastic', 'Sarcastic'], rotation=0)

# 2. Headline length distribution
df['headline_length'] = df['headline'].str.len()
df['headline_length'].hist(bins=50, ax=axes[0,1], color='lightgreen', alpha=0.7)
axes[0,1].set_title('Headline Length Distribution')
axes[0,1].set_xlabel('Length (characters)')
axes[0,1].set_ylabel('Frequency')

# 3. Word count distribution
df['word_count'] = df['headline'].str.split().str.len()
df['word_count'].hist(bins=30, ax=axes[1,0], color='salmon', alpha=0.7)
axes[1,0].set_title('Word Count Distribution')
axes[1,0].set_xlabel('Number of Words')
axes[1,0].set_ylabel('Frequency')

# 4. Length comparison by sarcasm
df.boxplot(column='headline_length', by='is_sarcastic', ax=axes[1,1])
axes[1,1].set_title('Headline Length by Sarcasm')
axes[1,1].set_xlabel('Is Sarcastic')
axes[1,1].set_ylabel('Length (characters)')

plt.tight_layout()
plt.show()

# Statistics
print("\nHeadline Length Statistics:")
print(df.groupby('is_sarcastic')['headline_length'].describe())

print("\nWord Count Statistics:")
print(df.groupby('is_sarcastic')['word_count'].describe())

## 2. Text Preprocessing

In [ ]:
# Text preprocessing functions
def clean_text(text):
    """
    Clean and preprocess text data
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove special characters but keep punctuation that might be important for sarcasm
    text = re.sub(r'[^\w\s\.,!?;:\'\"-]', ' ', text)
    
    # Remove extra spaces
    text = text.strip()
    
    return text

# Apply text cleaning
df['cleaned_headline'] = df['headline'].apply(clean_text)

# Show examples of cleaned text
print("Original vs Cleaned Headlines:")
for i in range(5):
    print(f"Original: {df['headline'].iloc[i]}")
    print(f"Cleaned:  {df['cleaned_headline'].iloc[i]}")
    print("-" * 80)

# Prepare data for modeling
X = df['cleaned_headline'].values
y = df['is_sarcastic'].values

print(f"\nFinal dataset shape: X={X.shape}, y={y.shape}")
print(f"Class distribution: {Counter(y)}")

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Training class distribution: {Counter(y_train)}")
print(f"Test class distribution: {Counter(y_test)}")

# Create validation split from training data
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"\nAfter validation split:")
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

## 3. PyTorch GRU Model Implementation

In [ ]:
# PyTorch tokenization and dataset preparation
class TextTokenizer:
    def __init__(self, max_vocab_size=10000):
        self.max_vocab_size = max_vocab_size
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.vocab_size = 0
        
    def fit(self, texts):
        # Count word frequencies
        word_freq = Counter()
        for text in texts:
            word_freq.update(text.split())
        
        # Create vocabulary with most frequent words
        self.word_to_idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx_to_word = {0: '<PAD>', 1: '<UNK>'}
        
        for word, freq in word_freq.most_common(self.max_vocab_size - 2):
            idx = len(self.word_to_idx)
            self.word_to_idx[word] = idx
            self.idx_to_word[idx] = word
            
        self.vocab_size = len(self.word_to_idx)
        print(f"Vocabulary size: {self.vocab_size}")
        
    def transform(self, texts):
        sequences = []
        for text in texts:
            sequence = [self.word_to_idx.get(word, 1) for word in text.split()]
            sequences.append(sequence)
        return sequences
    
    def fit_transform(self, texts):
        self.fit(texts)
        return self.transform(texts)

# Initialize tokenizer and fit on training data
pytorch_tokenizer = TextTokenizer(max_vocab_size=10000)
X_train_seq = pytorch_tokenizer.fit_transform(X_train)
X_val_seq = pytorch_tokenizer.transform(X_val)
X_test_seq = pytorch_tokenizer.transform(X_test)

# Pad sequences
max_length = 50  # Maximum sequence length
def pad_sequences_pytorch(sequences, max_len, pad_value=0):
    padded = []
    for seq in sequences:
        if len(seq) > max_len:
            padded.append(seq[:max_len])
        else:
            padded.append(seq + [pad_value] * (max_len - len(seq)))
    return np.array(padded)

X_train_padded = pad_sequences_pytorch(X_train_seq, max_length)
X_val_padded = pad_sequences_pytorch(X_val_seq, max_length)
X_test_padded = pad_sequences_pytorch(X_test_seq, max_length)

print(f"PyTorch padded shapes:")
print(f"Train: {X_train_padded.shape}")
print(f"Validation: {X_val_padded.shape}")
print(f"Test: {X_test_padded.shape}")

In [ ]:
# PyTorch Dataset class
class SarcasmDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.LongTensor(texts)
        self.labels = torch.FloatTensor(labels)
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Create datasets
train_dataset = SarcasmDataset(X_train_padded, y_train)
val_dataset = SarcasmDataset(X_val_padded, y_val)
test_dataset = SarcasmDataset(X_test_padded, y_test)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# PyTorch GRU Model
class GRUSarcasmClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, num_layers=2, dropout=0.3):
        super(GRUSarcasmClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers, 
                         batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, 64)  # *2 for bidirectional
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        
        # GRU
        gru_out, hidden = self.gru(embedded)
        
        # Use the last output
        last_output = gru_out[:, -1, :]
        
        # Fully connected layers
        x = self.dropout(last_output)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return self.sigmoid(x).squeeze()

# Initialize PyTorch model
pytorch_model = GRUSarcasmClassifier(
    vocab_size=pytorch_tokenizer.vocab_size,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2,
    dropout=0.3
).to(device)

print("PyTorch Model Architecture:")
print(pytorch_model)
print(f"\nTotal parameters: {sum(p.numel() for p in pytorch_model.parameters()):,}")

In [ ]:
# PyTorch training function
def train_pytorch_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    best_val_acc = 0
    patience_counter = 0
    patience = 7
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for batch_texts, batch_labels in train_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_texts)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            predicted = (outputs > 0.5).float()
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_texts, batch_labels in val_loader:
                batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
                
                outputs = model(batch_texts)
                loss = criterion(outputs, batch_labels)
                
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()
        
        # Calculate metrics
        train_loss_avg = train_loss / len(train_loader)
        val_loss_avg = val_loss / len(val_loader)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total
        
        train_losses.append(train_loss_avg)
        val_losses.append(val_loss_avg)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        
        # Learning rate scheduling
        scheduler.step(val_loss_avg)
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_pytorch_gru_model.pth')
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
        
        print(f'Epoch [{epoch+1}/{epochs}], '
              f'Train Loss: {train_loss_avg:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss_avg:.4f}, Val Acc: {val_acc:.4f}')
    
    return train_losses, val_losses, train_accuracies, val_accuracies

# Train the PyTorch model
print("Training PyTorch GRU Model...")
pytorch_train_losses, pytorch_val_losses, pytorch_train_accs, pytorch_val_accs = train_pytorch_model(
    pytorch_model, train_loader, val_loader, epochs=30, lr=0.001
)

In [ ]:
# PyTorch model evaluation
def evaluate_pytorch_model(model, test_loader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for batch_texts, batch_labels in test_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            
            outputs = model(batch_texts)
            predicted = (outputs > 0.5).float()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
            all_probabilities.extend(outputs.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_labels), np.array(all_probabilities)

# Load best model and evaluate
pytorch_model.load_state_dict(torch.load('best_pytorch_gru_model.pth'))
pytorch_predictions, pytorch_true_labels, pytorch_probabilities = evaluate_pytorch_model(pytorch_model, test_loader)

# Calculate metrics
pytorch_accuracy = accuracy_score(pytorch_true_labels, pytorch_predictions)
pytorch_precision = precision_score(pytorch_true_labels, pytorch_predictions)
pytorch_recall = recall_score(pytorch_true_labels, pytorch_predictions)
pytorch_f1 = f1_score(pytorch_true_labels, pytorch_predictions)
pytorch_auc = roc_auc_score(pytorch_true_labels, pytorch_probabilities)

print("PyTorch GRU Model Results:")
print(f"Accuracy: {pytorch_accuracy:.4f}")
print(f"Precision: {pytorch_precision:.4f}")
print(f"Recall: {pytorch_recall:.4f}")
print(f"F1-Score: {pytorch_f1:.4f}")
print(f"AUC: {pytorch_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(pytorch_true_labels, pytorch_predictions, 
                          target_names=['Not Sarcastic', 'Sarcastic']))

## 4. TensorFlow/Keras GRU Model Implementation

In [ ]:
# TensorFlow tokenization and data preparation
max_vocab_size = 10000
max_length = 50

# Initialize tokenizer
tf_tokenizer = Tokenizer(num_words=max_vocab_size, oov_token='<OOV>')
tf_tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_tf = tf_tokenizer.texts_to_sequences(X_train)
X_val_tf = tf_tokenizer.texts_to_sequences(X_val)
X_test_tf = tf_tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_tf = pad_sequences(X_train_tf, maxlen=max_length, padding='post')
X_val_tf = pad_sequences(X_val_tf, maxlen=max_length, padding='post')
X_test_tf = pad_sequences(X_test_tf, maxlen=max_length, padding='post')

print(f"TensorFlow tokenizer vocabulary size: {len(tf_tokenizer.word_index) + 1}")
print(f"TensorFlow padded shapes:")
print(f"Train: {X_train_tf.shape}")
print(f"Validation: {X_val_tf.shape}")
print(f"Test: {X_test_tf.shape}")

# Convert labels to numpy arrays
y_train_tf = np.array(y_train)
y_val_tf = np.array(y_val)
y_test_tf = np.array(y_test)

In [ ]:
# TensorFlow GRU Model
def create_gru_model(vocab_size, embedding_dim=128, gru_units=128, dropout_rate=0.3):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout_rate)),
        Bidirectional(GRU(gru_units//2, dropout=dropout_rate)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Create TensorFlow model
tf_model = create_gru_model(
    vocab_size=len(tf_tokenizer.word_index) + 1,
    embedding_dim=128,
    gru_units=128,
    dropout_rate=0.3
)

# Compile model
tf_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("TensorFlow Model Architecture:")
tf_model.summary()

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
    ModelCheckpoint('best_tensorflow_gru_model.keras', monitor='val_accuracy', 
                   save_best_only=True, save_weights_only=False)
]

# Train the model
print("\nTraining TensorFlow GRU Model...")
tf_history = tf_model.fit(
    X_train_tf, y_train_tf,
    validation_data=(X_val_tf, y_val_tf),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# TensorFlow model evaluation
# Load best model
tf_model = tf.keras.models.load_model('best_tensorflow_gru_model.keras')

# Make predictions
tf_predictions_prob = tf_model.predict(X_test_tf)
tf_predictions = (tf_predictions_prob > 0.5).astype(int).flatten()

# Calculate metrics
tf_accuracy = accuracy_score(y_test_tf, tf_predictions)
tf_precision = precision_score(y_test_tf, tf_predictions)
tf_recall = recall_score(y_test_tf, tf_predictions)
tf_f1 = f1_score(y_test_tf, tf_predictions)
tf_auc = roc_auc_score(y_test_tf, tf_predictions_prob)

print("TensorFlow GRU Model Results:")
print(f"Accuracy: {tf_accuracy:.4f}")
print(f"Precision: {tf_precision:.4f}")
print(f"Recall: {tf_recall:.4f}")
print(f"F1-Score: {tf_f1:.4f}")
print(f"AUC: {tf_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test_tf, tf_predictions, 
                          target_names=['Not Sarcastic', 'Sarcastic']))

# Training and validation accuracy/loss
train_acc_tf = tf_history.history['accuracy']
val_acc_tf = tf_history.history['val_accuracy']
train_loss_tf = tf_history.history['loss']
val_loss_tf = tf_history.history['val_loss']

## 5. Hyperparameter Tuning with Keras Tuner

In [ ]:
# Hyperparameter tuning with Keras Tuner
def build_tuner_model(hp):
    model = Sequential()
    
    # Embedding layer
    model.add(Embedding(
        input_dim=len(tf_tokenizer.word_index) + 1,
        output_dim=hp.Int('embedding_dim', min_value=64, max_value=256, step=32),
        input_length=max_length
    ))
    
    # First GRU layer
    model.add(Bidirectional(GRU(
        hp.Int('gru_units_1', min_value=64, max_value=256, step=32),
        return_sequences=True,
        dropout=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)
    )))
    
    # Second GRU layer
    model.add(Bidirectional(GRU(
        hp.Int('gru_units_2', min_value=32, max_value=128, step=16),
        dropout=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)
    )))
    
    # Dense layers
    model.add(Dense(
        hp.Int('dense_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Initialize tuner
tuner = kt.RandomSearch(
    build_tuner_model,
    objective='val_accuracy',
    max_trials=10,  # Reduce for faster execution
    directory='keras_tuner',
    project_name='gru_sarcasm_detection'
)

print("Tuner search space summary:")
tuner.search_space_summary()

# Search for best hyperparameters
print("\nStarting hyperparameter search...")
tuner.search(
    X_train_tf, y_train_tf,
    epochs=10,  # Reduce epochs for faster tuning
    validation_data=(X_val_tf, y_val_tf),
    batch_size=64,
    verbose=1
)

# Get best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("\nBest hyperparameters:")
for param in ['embedding_dim', 'gru_units_1', 'gru_units_2', 'dense_units', 
              'dropout_1', 'dropout_2', 'dropout_3', 'learning_rate']:
    print(f"{param}: {best_hp.get(param)}")

In [ ]:
# Train the best model with optimal hyperparameters
best_model = tuner.hypermodel.build(best_hp)

# Extended training with best hyperparameters
callbacks_tuned = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    ModelCheckpoint('best_tuned_gru_model.keras', monitor='val_accuracy', 
                   save_best_only=True, save_weights_only=False)
]

print("Training tuned model with extended epochs...")
tuned_history = best_model.fit(
    X_train_tf, y_train_tf,
    validation_data=(X_val_tf, y_val_tf),
    epochs=50,
    batch_size=64,
    callbacks=callbacks_tuned,
    verbose=1
)

# Evaluate tuned model
best_model = tf.keras.models.load_model('best_tuned_gru_model.keras')
tuned_predictions_prob = best_model.predict(X_test_tf)
tuned_predictions = (tuned_predictions_prob > 0.5).astype(int).flatten()

# Calculate metrics for tuned model
tuned_accuracy = accuracy_score(y_test_tf, tuned_predictions)
tuned_precision = precision_score(y_test_tf, tuned_predictions)
tuned_recall = recall_score(y_test_tf, tuned_predictions)
tuned_f1 = f1_score(y_test_tf, tuned_predictions)
tuned_auc = roc_auc_score(y_test_tf, tuned_predictions_prob)

print("\nTuned TensorFlow GRU Model Results:")
print(f"Accuracy: {tuned_accuracy:.4f}")
print(f"Precision: {tuned_precision:.4f}")
print(f"Recall: {tuned_recall:.4f}")
print(f"F1-Score: {tuned_f1:.4f}")
print(f"AUC: {tuned_auc:.4f}")

## 6. Model Visualization and Analysis

In [ ]:
# Comprehensive Model Performance Visualization

# 1. Training History Plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# PyTorch Training History
epochs_pytorch = range(1, len(pytorch_train_losses) + 1)
axes[0, 0].plot(epochs_pytorch, pytorch_train_losses, 'b-', label='Training Loss', linewidth=2)
axes[0, 0].plot(epochs_pytorch, pytorch_val_losses, 'r-', label='Validation Loss', linewidth=2)
axes[0, 0].set_title('PyTorch GRU - Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(epochs_pytorch, pytorch_train_accs, 'b-', label='Training Accuracy', linewidth=2)
axes[0, 1].plot(epochs_pytorch, pytorch_val_accs, 'r-', label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('PyTorch GRU - Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epochs')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# TensorFlow Training History
epochs_tf = range(1, len(train_loss_tf) + 1)
axes[0, 2].plot(epochs_tf, train_loss_tf, 'g-', label='Training Loss', linewidth=2)
axes[0, 2].plot(epochs_tf, val_loss_tf, 'orange', label='Validation Loss', linewidth=2)
axes[0, 2].set_title('TensorFlow GRU - Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Epochs')
axes[0, 2].set_ylabel('Loss')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(epochs_tf, train_acc_tf, 'g-', label='Training Accuracy', linewidth=2)
axes[1, 0].plot(epochs_tf, val_acc_tf, 'orange', label='Validation Accuracy', linewidth=2)
axes[1, 0].set_title('TensorFlow GRU - Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epochs')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Tuned Model Training History
if 'tuned_history' in locals():
    epochs_tuned = range(1, len(tuned_history.history['loss']) + 1)
    axes[1, 1].plot(epochs_tuned, tuned_history.history['loss'], 'purple', label='Training Loss', linewidth=2)
    axes[1, 1].plot(epochs_tuned, tuned_history.history['val_loss'], 'pink', label='Validation Loss', linewidth=2)
    axes[1, 1].set_title('Tuned GRU - Training & Validation Loss', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    axes[1, 2].plot(epochs_tuned, tuned_history.history['accuracy'], 'purple', label='Training Accuracy', linewidth=2)
    axes[1, 2].plot(epochs_tuned, tuned_history.history['val_accuracy'], 'pink', label='Validation Accuracy', linewidth=2)
    axes[1, 2].set_title('Tuned GRU - Training & Validation Accuracy', fontsize=14, fontweight='bold')
    axes[1, 2].set_xlabel('Epochs')
    axes[1, 2].set_ylabel('Accuracy')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Tuned Model\nTraining History\nNot Available', 
                    ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    axes[1, 2].text(0.5, 0.5, 'Tuned Model\nTraining History\nNot Available', 
                    ha='center', va='center', transform=axes[1, 2].transAxes, fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Confusion Matrices and ROC Curves
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Confusion Matrices
models_data = [
    ('PyTorch GRU', pytorch_true_labels, pytorch_predictions),
    ('TensorFlow GRU', y_test_tf, tf_predictions),
]

if 'tuned_predictions' in locals():
    models_data.append(('Tuned GRU', y_test_tf, tuned_predictions))

for i, (model_name, y_true, y_pred) in enumerate(models_data):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, i],
                xticklabels=['Not Sarcastic', 'Sarcastic'],
                yticklabels=['Not Sarcastic', 'Sarcastic'])
    axes[0, i].set_title(f'{model_name} - Confusion Matrix', fontsize=14, fontweight='bold')
    axes[0, i].set_xlabel('Predicted')
    axes[0, i].set_ylabel('Actual')

# ROC Curves
roc_data = [
    ('PyTorch GRU', pytorch_true_labels, pytorch_probabilities, pytorch_auc),
    ('TensorFlow GRU', y_test_tf, tf_predictions_prob.flatten(), tf_auc),
]

if 'tuned_predictions_prob' in locals():
    roc_data.append(('Tuned GRU', y_test_tf, tuned_predictions_prob.flatten(), tuned_auc))

# Plot all ROC curves together
for model_name, y_true, y_prob, auc_score in roc_data:
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    axes[1, 0].plot(fpr, tpr, linewidth=2, label=f'{model_name} (AUC = {auc_score:.3f})')

axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('ROC Curves Comparison', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Model Comparison Bar Chart
model_names = ['PyTorch GRU', 'TensorFlow GRU']
accuracies = [pytorch_accuracy, tf_accuracy]
f1_scores = [pytorch_f1, tf_f1]
auc_scores = [pytorch_auc, tf_auc]

if 'tuned_accuracy' in locals():
    model_names.append('Tuned GRU')
    accuracies.append(tuned_accuracy)
    f1_scores.append(tuned_f1)
    auc_scores.append(tuned_auc)

x = np.arange(len(model_names))
width = 0.25

axes[1, 1].bar(x - width, accuracies, width, label='Accuracy', alpha=0.8)
axes[1, 1].bar(x, f1_scores, width, label='F1-Score', alpha=0.8)
axes[1, 1].bar(x + width, auc_scores, width, label='AUC', alpha=0.8)

axes[1, 1].set_xlabel('Models')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(model_names)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(0, 1)

# Add value labels on bars
for i, (acc, f1, auc) in enumerate(zip(accuracies, f1_scores, auc_scores)):
    axes[1, 1].text(i - width, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
    axes[1, 1].text(i, f1 + 0.01, f'{f1:.3f}', ha='center', va='bottom', fontweight='bold')
    axes[1, 1].text(i + width, auc + 0.01, f'{auc:.3f}', ha='center', va='bottom', fontweight='bold')

# Metrics Summary Table
metrics_data = {
    'Model': model_names,
    'Accuracy': [f'{acc:.4f}' for acc in accuracies],
    'Precision': [f'{pytorch_precision:.4f}', f'{tf_precision:.4f}'] + 
                 ([f'{tuned_precision:.4f}'] if 'tuned_precision' in locals() else []),
    'Recall': [f'{pytorch_recall:.4f}', f'{tf_recall:.4f}'] + 
              ([f'{tuned_recall:.4f}'] if 'tuned_recall' in locals() else []),
    'F1-Score': [f'{f1:.4f}' for f1 in f1_scores],
    'AUC': [f'{auc:.4f}' for auc in auc_scores]
}

axes[1, 2].axis('tight')
axes[1, 2].axis('off')
table = axes[1, 2].table(cellText=[list(row) for row in zip(*[metrics_data[col] for col in metrics_data.keys()])],
                        colLabels=list(metrics_data.keys()),
                        cellLoc='center',
                        loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 2)
axes[1, 2].set_title('Model Performance Summary', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## 7. Results Summary and Conclusions

In [ ]:
# Final Results Summary
print("="*80)
print("FINAL RESULTS SUMMARY - SARCASM DETECTION WITH GRU MODELS")
print("="*80)

print("\n📊 DATASET OVERVIEW:")
print(f"• Total samples: {len(df):,}")
print(f"• Training samples: {len(X_train):,}")
print(f"• Validation samples: {len(X_val):,}")
print(f"• Test samples: {len(X_test):,}")
print(f"• Class distribution: {Counter(y)}")

print("\n🔥 MODEL PERFORMANCE COMPARISON:")
print("-" * 50)

# Create comparison table
comparison_data = {
    'Model': ['PyTorch GRU', 'TensorFlow GRU'],
    'Accuracy': [f'{pytorch_accuracy:.4f}', f'{tf_accuracy:.4f}'],
    'Precision': [f'{pytorch_precision:.4f}', f'{tf_precision:.4f}'],
    'Recall': [f'{pytorch_recall:.4f}', f'{tf_recall:.4f}'],
    'F1-Score': [f'{pytorch_f1:.4f}', f'{tf_f1:.4f}'],
    'AUC-ROC': [f'{pytorch_auc:.4f}', f'{tf_auc:.4f}']
}

if 'tuned_accuracy' in locals():
    comparison_data['Model'].append('Tuned GRU')
    comparison_data['Accuracy'].append(f'{tuned_accuracy:.4f}')
    comparison_data['Precision'].append(f'{tuned_precision:.4f}')
    comparison_data['Recall'].append(f'{tuned_recall:.4f}')
    comparison_data['F1-Score'].append(f'{tuned_f1:.4f}')
    comparison_data['AUC-ROC'].append(f'{tuned_auc:.4f}')

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\n🎯 ACHIEVEMENT ANALYSIS:")
print("-" * 30)

# Check if models meet the 80% accuracy requirement
target_accuracy = 0.80
models_performance = [
    ('PyTorch GRU', pytorch_accuracy),
    ('TensorFlow GRU', tf_accuracy)
]

if 'tuned_accuracy' in locals():
    models_performance.append(('Tuned GRU', tuned_accuracy))

for model_name, accuracy in models_performance:
    status = "✅ PASSED" if accuracy >= target_accuracy else "❌ FAILED"
    print(f"• {model_name}: {accuracy:.4f} ({accuracy*100:.2f}%) - {status}")

print(f"\n📈 TARGET REQUIREMENTS STATUS:")
print("-" * 35)
print("✅ Deep Learning Models: PyTorch & TensorFlow GRU models implemented")
print("✅ Evaluation Metrics: Accuracy, Precision, Recall, F1-Score, AUC-ROC calculated")
print("✅ Visualizations: Training curves, confusion matrices, ROC curves created")
print("✅ Hyperparameter Tuning: Keras Tuner optimization performed")

best_model = max(models_performance, key=lambda x: x[1])
print(f"\n🏆 BEST MODEL: {best_model[0]} with {best_model[1]:.4f} ({best_model[1]*100:.2f}%) accuracy")

if best_model[1] >= target_accuracy:
    print("🎉 SUCCESS: Target accuracy of 80% achieved!")
else:
    print("⚠️  NOTE: Target accuracy of 80% not achieved. Consider:")
    print("   - Increasing model complexity")
    print("   - Additional data preprocessing")
    print("   - Extended hyperparameter tuning")
    print("   - Data augmentation techniques")

print("\n💡 KEY INSIGHTS:")
print("-" * 15)
print("• GRU models effectively capture sequential patterns in text for sarcasm detection")
print("• Bidirectional GRU layers improve performance by processing text in both directions")
print("• Hyperparameter tuning can significantly improve model performance")
print("• Both PyTorch and TensorFlow implementations show comparable results")
print("• The dataset shows balanced class distribution, which is good for training")

print("\n🔧 TECHNICAL IMPLEMENTATION:")
print("-" * 28)
print("• Text preprocessing: Cleaning, tokenization, and padding")
print("• Model architecture: Embedding → Bidirectional GRU → Dense layers")
print("• Training strategies: Early stopping, learning rate scheduling")
print("• Evaluation: Comprehensive metrics and visualization")

print("="*80)

## 8. Google Colab Instructions and Tips

### 🚀 Running on Google Colab with GPU/TPU

To achieve optimal performance as suggested in the requirements, follow these steps:

#### **1. Enable GPU/TPU in Google Colab:**
```python
# Check GPU availability
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# For TPU (if available)
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU Available: ", strategy.num_replicas_in_sync)
except:
    print("TPU not available")
```

#### **2. Install Required Packages:**
```bash
!pip install keras-tuner
!pip install wordcloud
```

#### **3. Upload Dataset:**
```python
from google.colab import files
uploaded = files.upload()  # Upload DeteksiSarkasme.json
```

#### **4. Adjust Batch Size for GPU:**
- Increase batch size to 128 or 256 for better GPU utilization
- Use mixed precision training for faster computation:

```python
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
```

### 📈 Performance Optimization Tips

1. **Data Loading**: Use `tf.data.Dataset` for efficient data pipeline
2. **Model Parallelism**: Utilize multiple GPUs if available
3. **Gradient Accumulation**: For large batch simulation with limited memory
4. **Learning Rate Scheduling**: Use cosine annealing or cyclical learning rates

### 🎯 Expected Results
With proper GPU/TPU acceleration, you should expect:
- **Training Time**: 15-30 minutes per model
- **Target Accuracy**: ≥80% on both training and testing sets
- **Hyperparameter Tuning**: 10-20 trials in reasonable time

### 🔧 Troubleshooting Common Issues

**Memory Issues:**
- Reduce batch size
- Use gradient checkpointing
- Clear cache between models

**Performance Issues:**
- Ensure GPU is properly utilized
- Check data loading pipeline
- Use TensorFlow profiler for bottleneck analysis